# 1 Chains

在 LangChain 框架中，链（Chains）是实现任务自动化的核心机制。随着框架从 0.x 版本升级到 1.x，链的设计理念也发生了重要演变 —— 从传统的类封装模式转向更灵活的 LangChain Expression Language（LCEL）表达式模式。本文将系统梳理当前主流的链类型，分析每种类型的适用场景，并通过实践案例展示其应用方法。

## 一、链类型的技术演进与分类框架

LangChain 的链类型发展可分为两个阶段：传统链（Traditional Chains）和现代 LCEL 链（LangChain Expression Language Chains）。这种演进源于对灵活性和可组合性的追求：

- **传统链**：基于类封装的预定义工作流，如LLMChain、SequentialChain等，在 LangChain 0.2.x 后部分已标记为废弃。

- **LCEL 链**：通过|运算符组合Runnable对象构建的工作流，支持流式处理、异步执行和并行计算，是 1.x 版本的推荐方案。

这种演变带来的核心优势是：开发者可以像搭积木一样组合不同组件，而非局限于预定义的链结构。接下来我们将重点介绍仍被广泛使用的传统链和现代 LCEL 链的主要类型。

## 二、核心链类型详解与适用场景

**（一）传统链类型（兼容保留、不详细介绍）**

尽管 LCEL 已成为主流，传统链类型仍在一些 legacy 项目中使用，了解它们有助于理解链设计的基本原理。

1. **LLMChain**

    - 核心功能：最基础的链类型，将提示模板（PromptTemplate）与语言模型（LLM）封装为单一组件。

    - 适用场景：简单的单步任务，如文本生成、基础问答、格式转换等。

    - 局限性：仅支持单输入单输出，已被 LCEL 的prompt | llm | parser模式替代。

2. **SimpleSequentialChain**

    - 核心功能：按线性顺序串联多个链，前一个链的输出作为后一个链的唯一输入。

    - 适用场景：单一数据流的多步骤处理，例如 "文本翻译→翻译结果总结" 的流水线任务。

    - 优势：实现简单，适合逻辑清晰的线性流程。

3. **SequentialChain**

    - 核心功能：支持多输入多输出的复杂线性流程，允许不同链共享或传递多个变量。

    - 适用场景：需要处理多源数据或生成多维度结果的任务，如电商评论分析中的 "翻译→情感分析→标签提取→报告生成" 流程。

    - 与 SimpleSequentialChain 的区别：后者仅支持单一输入输出，前者可处理复杂变量传递。

4. **MapReduceChain**

    - 核心功能：采用 "分块处理→结果合并" 模式处理长文档，先将文本分割为小块并行处理，再汇总分析。

    - 适用场景：长文档总结、学术论文分析、多章节报告生成等场景。

    - 优势：解决了大模型上下文长度限制问题，提高长文本处理效率。

5. **TransformChain**

    - 核心功能：允许插入自定义 Python 函数对数据进行预处理或后处理。

    - 适用场景：文本清洗、格式转换、特定字段提取、数据过滤等需要编程逻辑的任务。

    - 典型案例：从 HTML 文本中提取正文内容、按规则截取长文本的关键段落。

**（二）现代 LCEL 链类型（1.x 推荐，详细介绍）**

LCEL（LangChain Expression Language）引入了Runnable接口，通过 `|` 运算符实现组件组合，提供了更高的灵活性和扩展性。

1. RunnableSequence

    - 核心功能：按顺序执行多个Runnable组件，前一个输出作为后一个输入，对应传统的SequentialChain。

    - 语法特点：使用 `|` 运算符组合，如`prompt | llm | parser`。

    - 适用场景：所有线性工作流，尤其适合需要精细控制每个步骤的场景。

2. RunnableParallel

    - 核心功能：并行执行多个Runnable组件，将结果合并为字典输出。

    - 适用场景：需要同时处理多个独立任务的场景，如 "文本总结 + 关键词提取 + 情感分析" 并行执行。

    - 性能优势：比串行执行节省大量时间，尤其适合多文档处理场景。

3. RunnableBranch

    - 核心功能：根据条件判断选择不同的处理分支，类似程序中的if-elif-else逻辑。

    - 适用场景：需要动态路由的任务，如客服系统中根据问题类型（技术支持 / 订单查询 / 投诉建议）分配不同处理流程。

4. RunnableWithMessageHistory

    - 核心功能：自动管理对话历史，维持上下文连贯性。
    
    - 适用场景：聊天机器人、多轮对话系统等需要上下文记忆的场景。

## 三、实践案例一：LCEL 链实现多步骤文本处理

### 案例1：基础单链 ———— 生成产品描述

- **适用场景**：简单的单步文本生成任务（如广告语、摘要、基础问答）

- **链类型**： 最基础的“提示词模板 + LLM”组合

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.7,
)

# 1. 定义提示词模板
prompt = ChatPromptTemplate.from_template("""
你是电商平台的文案专家，请为以下产品生成3条吸引人的短标题（每条不超过20字）：
产品：{product}
特点：{features}
""")

# 2. 构建基础链（提示 > 模型 > 输出解析）
product_chain = prompt | llm | StrOutputParser()

# 3. 运行链
result = product_chain.invoke({
    "product": "便携式蓝牙音箱",
    "features": "防水、20小时续航、360度环绕音效"
})

print(result)


1. 海滩派对必备！防水蓝牙音箱20小时续航
2. 轻巧便携，360度环绕音效震撼全场
3. 水中音乐无压力，蓝牙音箱带你畅游


在 LangChain 中，StrOutputParser是最常用的输出解析器之一，它的核心作用是将语言模型返回的原始输出（通常是AIMessage对象）转换为纯字符串格式，方便后续处理或直接使用。

简单来说，当你调用语言模型时，模型返回的结果并不是直接可用的字符串，而是包含了消息类型、内容等信息的对象。StrOutputParser就像一个 "转换器"，专门提取其中的文本内容并以字符串形式返回。

### 案例2：线性链 ———— 翻译 → 总结 → 扩写

- **适用场景**：多步骤线性流程（前一步输出作为后一步输入）

- **链类型**：`RunnableSequence` （LCEL的线性链，替代传统的`SequentialChain`）

In [7]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.3,
)

# 步骤1：将英文翻译成中文
translate_prompt = ChatPromptTemplate.from_template("""
将以下的英文文本翻译成流畅的中文：
{english_text}
中文翻译：
""")
translate_chain = translate_prompt | llm | StrOutputParser()

# 步骤2：总结中文内容
summary_prompt = ChatPromptTemplate.from_template("""
总结以下中文文本的核心观点（不超过50字）：
{chinese_text}
总结：
""")
summary_chain = summary_prompt | llm | StrOutputParser()

# 步骤3：基于总结扩写成详细说明
expand_prompt = ChatPromptTemplate.from_template("""
将以下总结扩写成300字左右的详细说明：
{summary_text}
详细说明：
""")
expand_chain = expand_prompt | llm | StrOutputParser()

# 步骤4：构建线性链
total_chain = RunnableSequence(
    # 第一步：翻译，输出到chinese_text变量
    lambda x: {"chinese_text": translate_chain.invoke(x)},
    # 第二步：总结，用chinese_text作为输入，输出到summary_text变量
    lambda x: {"summary_text": summary_chain.invoke({"chinese_text": x["chinese_text"]})},
    # 第三步：扩写，用summary_text作为输入
    expand_chain
)

# 运行链
english_article = """
LangChain 1.0 introduces significant improvements in flexibility and performance. 
The new LCEL (LangChain Expression Language) allows developers to compose 
components more intuitively using the | operator, supporting streaming, 
parallel execution, and async operations.
"""

result = total_chain.invoke({"english_text": english_article})
print(result)


LangChain 1.0 的发布标志着该框架在灵活性和性能方面的显著提升。新引入的 LCEL（LangChain Expression Language）语言，极大地简化了开发者在构建复杂应用时的工作流程。通过 LCEL，开发者可以更加直观地使用运算符来组合不同的组件，从而实现更高效的代码编写和逻辑表达。这种表达方式不仅易于理解，还能减少代码的冗余，提高可读性。

此外，LangChain 1.0 还支持流式处理、并行执行和异步操作，这些功能的引入使得处理大规模数据和复杂任务变得更加高效。在流式处理方面，开发者可以实时处理数据流，减少延迟，提高响应速度。而并行执行则允许多个任务同时进行，充分利用系统资源，显著提升了整体性能。异步操作的支持则使得开发者能够在等待某些操作完成时，继续执行其他任务，从而提高了应用的响应能力和用户体验。

这些改进使得 LangChain 1.0 不仅适用于简单的应用场景，更能够满足复杂系统的需求，适应不断变化的技术环境。开发者可以利用这些新特性，构建出更为灵活、高效的应用程序，推动业务的创新与发展。总之，LangChain 1.0 的推出为开发者提供了更强大的工具，助力他们在构建智能应用时，能够更加高效地实现目标。


- 线性链的关键是**变量传递**：前一步的输出（如`chinese_text`需作为后一步的输入变量）

- 支持 lambda 函数作为中间处理结果

- 适合有明确先后顺序的任务，每一步都依赖上一步的结果

### 案例3：并行链 ———— 多维度内容分析

- **适用场景**：需要同时执行多个独立任务（如多角度分析同一个文本）

- **链类型**： `RunnableParallel`

In [8]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.2,
)

# 1. 情感分析链
sentiment_prompt = ChatPromptTemplate.from_template("""
分析以下用户评论的情感倾向（输出：正面/负面/中性）：
{review}
情感分析：
""")
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

# 2. 关键词提取链
keyword_prompt = ChatPromptTemplate.from_template("""
从以下用户评论中提取最多5个核心的关键词：
{review}
关键词：
""")
keyword_chain = keyword_prompt | llm | StrOutputParser()

# 3. 建议生成链
suggestion_prompt = ChatPromptTemplate.from_template("""
基于以下用户评论，为商家提出1条改进建议：
{review}
改进建议：
""")
suggestion_chain = suggestion_prompt | llm | StrOutputParser()

# 构建并行链（同时执行3个独立任务）
parallel_chain = RunnableParallel(
    sentiment=sentiment_chain,
    keywords=keyword_chain,
    suggestion=suggestion_chain
)

# 运行链（单输入同时喂给3个链）
user_review = """
这款无线耳机续航不错，充一次能用10小时，但音质一般，尤其是低音部分比较弱。
另外佩戴久了耳朵会有点疼，希望能改进耳塞的材质。
"""

result = parallel_chain.invoke({"review": user_review})

print(result)


{'sentiment': '这条评论是中性的。用户对无线耳机的续航能力表示满意，但对音质和佩戴舒适度有一些不满，希望能有改进。整体来说，评论中既有正面的评价也有负面的评价，因此可以判断为中性情感倾向。', 'keywords': '1. 无线耳机\n2. 续航\n3. 音质\n4. 低音\n5. 耳塞', 'suggestion': '改进建议：考虑提升耳机的音质表现，特别是低音部分，可以通过优化驱动单元或调音来增强低频效果。同时，针对佩戴舒适度问题，建议使用更柔软、透气的耳塞材质，以减少长时间佩戴带来的不适感。'}


- 并行链的优势是**效率提升**：3个任务同时执行，总耗时约等于单个任务的耗时

- 是个对同一个输入进行多维度分析

- 注意：并行任务必须是相互独立的。

### 案例4：分支链 ———— 智能客服路由

- **适用场景**：根据输入特征动态选择处理逻辑（如客服系统的问题分类处理）

- **链类型**： `RunnableBranch`

In [11]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.1,
)

# 定义3种问题类型的处理链
# 1. 订单查询处理链
order_prompt = ChatPromptTemplate.from_template("""
你现在需要处理用户的订单查询：
用户问题：{query}
请回复："已为您查询订单状态，您的订单{query}目前处于[处理中]，预计24小时内发货。"
""")
order_chain = order_prompt | llm | StrOutputParser()

# 2. 退款处理链
refund_prompt = ChatPromptTemplate.from_template("""
你现在需要处理用户的退款请求：
用户问题：{query}
请回复："已收到您的退款申请'{query}'，将在3个工作日内审核完成，结果将通过短信通知。"
""")
refund_chain = refund_prompt | llm | StrOutputParser()

## 3. 其他问题处理链（默认分支）
other_prompt = ChatPromptTemplate.from_template("""
你现在需要回复用户的其他问题：
用户问题：{query}
请礼貌回复："感谢您的咨询，关于'{query}'，我们将安排专属客服在1小时内与您联系。"
""")
other_chain = other_prompt | llm | StrOutputParser()

# 构建分支链（根据问题类型路由到不同处理链）
branch_chain = RunnableBranch(
    # 条件1：包含“订单”关键词
    (lambda x: "订单" in x["query"], order_chain),
    # 条件2：包含“退款”关键词
    (lambda x: "退款" in x["query"], refund_chain),
    # 默认分支：其他问题
    other_chain
)

# 测试不同类型的问题
test_queries = [
    "我的订单12345什么时候发货？",
    "我想退款，因为收到的商品有破损",
    "你们支持7天无理由退货吗？"
]

for query in test_queries:
    print(f"用户问题：{query}")
    print(f"回复：{branch_chain.invoke({'query': query})}\n")



用户问题：我的订单12345什么时候发货？
回复：已为您查询订单状态，您的订单12345目前处于[处理中]，预计24小时内发货。

用户问题：我想退款，因为收到的商品有破损
回复：已收到您的退款申请'我想退款，因为收到的商品有破损'，将在3个工作日内审核完成，结果将通过短信通知。

用户问题：你们支持7天无理由退货吗？
回复：感谢您的咨询，关于'你们支持7天无理由退货吗？'，我们将安排专属客服在1小时内与您联系。如果您有任何其他问题或需要帮助，请随时告诉我们。感谢您的支持！祝您生活愉快！



- 分支链的核心是**条件判断**：每个分支由（条件函数、处理链）组成

- 适合需要动态路由的场景

### 案例5：复杂组合链 ———— 长文本分析系统

- **适用场景**：处理超长文本（如论文、报告）的综合分析

- **链类型**：结合`RunnableParallel`、`RunnableSequence`、`TransformChain`功能

In [14]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableSequence
from langchain.text_splitter import CharacterTextSplitter

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.2
)

# 步骤1：文本分割（处理长文档）
def split_text(input_dict):
    text = input_dict["long_text"]
    # 按字符分割，每块500字符，重叠50字符
    text_splitter = CharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separator="\n"
    )
    chunks = text_splitter.split_text(text)
    return {"chunks": chunks}

# 步骤2：分块总结（对每个片段生成小结）
chunk_summary_prompt = ChatPromptTemplate.from_template("""
总结以下文档片段的核心内容：
{chunk}
小结：
""")
chunk_summary_chain = chunk_summary_prompt | llm | StrOutputParser()

# 步骤3：合并小结（生成全文总结）
merge_summary_prompt = ChatPromptTemplate.from_template("""
将以下多个文档小结合并为一篇连贯的全文总结：
{chunk_summaries}
全文总结：
""")
merge_summary_chain = merge_summary_prompt | llm | StrOutputParser()

# 步骤4：并行生成关键词和核心观点（基于全文总结）
keyword_chain = ChatPromptTemplate.from_template("""
从以下总结中提取10个核心关键词：
{full_summary}
关键词：
""") | llm | StrOutputParser()

key_point_chain = ChatPromptTemplate.from_template("""
从以下总结中提炼3个核心观点：
{full_summary}
核心观点：
""") | llm | StrOutputParser()

# 构建总链
total_chain = RunnableSequence(
    # 1. 分割文本
    split_text,
    # 2. 分块总结（对每个chunk应用chunk_summary_chain）
    lambda x: {"chunk_summaries": "\n".join([chunk_summary_chain.invoke({"chunk": c}) for c in x["chunks"]])},
    # 3. 合并小结生成全文总结
    lambda x: {"full_summary": merge_summary_chain.invoke(x)},
    # 4. 并行生成关键词和核心观点
    RunnableParallel(
        full_summary=lambda x: x["full_summary"],
        keywords=keyword_chain,
        key_points=key_point_chain
    )
)

# 测试长文本（模拟一篇技术文章）
long_text = """
LangChain 1.0的核心更新之一是引入了LangChain Expression Language（LCEL），这是一种用于组合组件的声明式语法。
LCEL基于Runnable接口，允许开发者使用|运算符连接不同组件，构建复杂的工作流。

与传统的链类型相比，LCEL提供了多项优势：首先是灵活性，开发者可以更直观地组合组件，无需记忆特定的链类；
其次是支持流式输出，这对构建实时交互应用至关重要；此外，LCEL原生支持异步操作和并行执行，
能显著提升应用性能。

在实际应用中，LCEL的常见模式包括：RunnableSequence用于线性流程，RunnableParallel用于并行任务，
RunnableBranch用于条件分支。这些模式可以相互嵌套，形成复杂的应用逻辑。例如，一个智能问答系统
可能先使用RunnableBranch判断问题类型，再用RunnableSequence执行检索-生成流程，同时用
RunnableParallel进行多源信息验证。

LCEL还简化了错误处理和日志记录，开发者可以通过添加中间件来捕获和处理组件执行中的异常，
这对于生产环境中的应用稳定性至关重要。随着LangChain生态的发展，LCEL正成为构建
LLM应用的标准方式。
"""

# 运行链
result = total_chain.invoke({"long_text": long_text})

# 输出结果
print("全文总结：\n", result["full_summary"], "\n")
print("关键词：\n", result["keywords"], "\n")
print("核心观点：\n", result["key_points"])


全文总结：
 LangChain 1.0引入了LangChain Expression Language（LCEL），一种声明式语法，基于Runnable接口，支持灵活性、流式输出、异步操作和并行执行，提升应用性能。常见模式包括RunnableSequence、RunnableParallel和RunnableBranch，可相互嵌套形成复杂应用逻辑。LCEL简化了错误处理和日志记录，允许开发者添加中间件来捕获和处理组件执行中的异常，增强应用在生产环境中的稳定性。随着LangChain生态的发展，LCEL正逐渐成为构建LLM应用的标准方式。 

关键词：
 1. LangChain 1.0  
2. LangChain Expression Language (LCEL)  
3. 声明式语法  
4. Runnable接口  
5. 流式输出  
6. 异步操作  
7. 并行执行  
8. 应用性能  
9. 错误处理  
10. 中间件   

核心观点：
 1. LangChain 1.0引入了LangChain Expression Language（LCEL），这是一种声明式语法，支持灵活性、流式输出、异步操作和并行执行，有助于提升应用性能。
2. LCEL包括常见模式如RunnableSequence、RunnableParallel和RunnableBranch，这些模式可以相互嵌套形成复杂的应用逻辑，同时简化了错误处理和日志记录。
3. LCEL允许开发者添加中间件来捕获和处理组件执行中的异常，从而增强应用在生产环境中的稳定性，随着LangChain生态的发展，LCEL正逐渐成为构建LLM应用的标准方式。
